<a href="https://colab.research.google.com/github/HumphreyChou/CLIC-2021-Learned-Image-Compression/blob/main/CLIC2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIC2021

此笔记本是私人的，你可以在此处运行代码，但是所有结果不会被保存，
选择：文件 > 在云端硬盘中保存一份副本 可以创建你自己的笔记本

使用时一定选择 修改 > 笔记本设置 > 硬件加速器 GPU

## 环境配置
挂载云端硬盘
将自己的云端硬盘挂载在 colab 虚拟机文件系统下，可以使用自己上传的代码并保存训练数据

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


切换到当前工作路径

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CLIC2021")

### 安装配置 Anaconda

In [ ]:
# !wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
!bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local
!conda update -y -q
!conda env list

In [ ]:
!conda install -y -q python=3.7 cudatoolkit=10.0 cudnn

In [ ]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.7/site-packages/")
!source /usr/local/etc/profile.d/conda.sh
!source ~/.bashrc

检查 python 版本，运行结果应该为 `Python 3.7.*`

In [ ]:
!python --version

Python 3.7.10


### 上传代码
此处使用修改后的 Google HiFiC 代码



In [ ]:
!git clone https://github.com/HumphreyChou/compression.git

### 安装第三方包

In [ ]:
%%shell
cd compression/models/

pip install -r hific/requirements.txt
pip install scipy

## 训练

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CLIC2021/compression/models")

In [ ]:
!ls

bls2017.py    hific				  ms2020.py  toy_sources
bmshj2018.py  lpips_weight__net-lin_alex_v0.1.pb  tfci.py


先训练 MSE + LPIPS 模型

In [ ]:
%%shell
python -m hific.train --config mselpips --ckpt_dir ../../ckpts/mse_lpips \
    --num_steps 1k --tfds_dataset_name cifar10 --no-image-summaries



INFO:tensorflow:Using TFDS=TFDSArguments(dataset_name='cifar10', features_key='image', downloads_dir=None)
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0% 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0% 0/1 [00:00<?, ? url/s]
Dl Size...:   0% 0/162 [00:00<?, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...:   0% 0/1 [00:00<?, ? url/s]
Dl Size...:   1% 1/162 [00:00<02:29,  1.08 MiB/s]

Dl Completed...:   0% 0/1 [00:01<?, ? url/s]
Dl Size...:   1% 2/162 [00:01<02:28,  1.08 MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Completed...:   0% 0/1 [00:01<?, ? url/s]
Dl Size...:   2% 3/162 [00:01<00:44,  3.57 MiB/s]

Dl Completed...:   0% 0/1 [00:01<?, ? url/s]
Dl Size...:   2% 4/162 [00:01<00:44,  3.57 MiB/s]

Dl Completed...:   0% 0/1 [00:01<?, ? url/s]
Dl Size...:   3% 5/162 [00:01<00:44,  3.57 MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Completed...: 

再训练 GAN 模型

In [ ]:
%%shell
python -m hific.train --config hific --ckpt_dir ../../ckpts/hific \
                --init_autoencoder_from_ckpt_dir ../../ckpts/mse_lpips \
                --num_steps 10k \
                --tfds_dataset_name cifar10 \
                --no-image-summaries


INFO:tensorflow:GAN Training enabled. Training discriminator for 1 steps.
INFO:tensorflow:Using TFDS=TFDSArguments(dataset_name='cifar10', features_key='image', downloads_dir=None)
2021-05-06 14:28:18.196106: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-06 14:28:18.330733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-06 14:28:18.331589: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2021-05-06 14:28:18.376164: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2021-05-06 14:28:18.385388: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully op

## 评估

生成压缩图像，计算 PSNR 与 bpp

In [ ]:
%%shell
python -m compression/models/hific.evaluate --config hific \
                   --ckpt_dir ../../ckpts/hific \
                   --out_dir ../../out \
                   --tfds_dataset_name cifar10 \
                   --max_image 100

计算 MS-SSIM

In [ ]:
%%shell

python compression/models/hific/msssim.py --original_image=original.png --compared_image=distorted.png

## 打扫战场
配置环境、训练过程中会占用硬盘空间，直到占满。colab运行在虚拟机上，且训练产生出的参数和 checkpoint 存储在 ckpt_dir（可以是你指定的云端磁盘位置），因此可以直接选择 代码执行程序 > 将该运行时恢复出厂设置，即可被分配新的虚拟机，和所有的烦恼说拜拜

实际上，被分配的虚拟机存活时间最长12小时，即12小时后一切配置都会重置，这是谷歌的动态分配策略，以保证免费资源供应